In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import matplotlib.image as mpimg
import glob, os
from pathlib import Path
from IPython.core.display import display,HTML
pd.set_option('display.max_colwidth', None)

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation-221126.csv')
labels

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN


In [4]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104)
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103)
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106)
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,POINT (-122.32072 47.66111)
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2533,Devon Snyder,2533,479,University District,Occlusion,47.659710,-122.319855,bWNcSsunJ_MyQcFwru1edg,2022-11-09 08:08:14.709-08,NaN,NaN,f,NaN,NaN,POINT (-122.31985 47.65971)
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972)
2017,2535,Devon Snyder,2535,479,University District,Occlusion,47.659653,-122.319908,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:36.919-08,NaN,NaN,f,NaN,NaN,POINT (-122.31991 47.65965)
2018,2536,Devon Snyder,2536,479,University District,SurfaceProblem,47.659668,-122.319916,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:40.146-08,NaN,2.0,f,cracks,NaN,POINT (-122.31992 47.65967)


In [5]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [16]:
#map_clusters(chicago,0)

In [43]:
#get curb ramp cluster dataset for chicago 
chicago_curb_ramp = cluster_label_type_at_index(chicago,0)[2]
#get problem cluster dataset for chicago 
chicago_problem = cluster_label_type_at_index(chicago,2)[2]
#get no curb cluster dataset for chicago 
chicago_nocurb = cluster_label_type_at_index(chicago,7)[2]
#get no sidewalk cluster dataset for chicago 
chicago_nosidewalk = cluster_label_type_at_index(chicago,1)[2]
#get occlusion cluster dataset for chicago 
chicago_occlusion = cluster_label_type_at_index(chicago,3)[2]
#get signal cluster dataset for chicago 
chicago_signal = cluster_label_type_at_index(chicago,9)[2]
#get crosswalk cluster dataset for chicago 
chicago_crosswalk = cluster_label_type_at_index(chicago,8)[2]

In [ ]:
# label_types = ['CurbRamp', 
#                'NoSidewalk', 
#                'Problem', 
#                'Occlusion', 
#                'SurfaceProblem', 
#                'Obstacle', 
#                'Other', 
#                'NoCurbRamp', 
#                'Crosswalk', 
#                'Signal']

In [7]:
# locate folder with images
chicago_folder ="chicago_with_icons/"

In [48]:
def append_img_url(df, folder):
    df['img_url'] = df.apply(lambda x: 'img/'+folder + "label_id_"+str(x.label_id)+".jpg", axis=1)
    return df.head()

In [49]:
append_img_url(chicago_curb_ramp, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
76,94,Devon Snyder,94,447,Skokie,CurbRamp,42.022606,-87.728157,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:14.398-07,t,4.0,f,"not enough landing space,narrow,not level with street",NaN,POINT (-87.72816 42.02261),2,img/chicago_with_icons/label_id_94.jpg
77,96,Devon Snyder,96,447,Skokie,CurbRamp,42.022720,-87.728149,rcjtiNG8gkNtkifZDFj2NQ,2022-05-10 14:56:58.603-07,t,3.0,f,"not enough landing space,surface problem",NaN,POINT (-87.72815 42.02272),1,img/chicago_with_icons/label_id_96.jpg
78,97,Devon Snyder,97,437,Skokie,CurbRamp,42.022717,-87.728432,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:57:43.381-07,t,3.0,f,"not enough landing space,points into traffic",NaN,POINT (-87.72843 42.02272),15,img/chicago_with_icons/label_id_97.jpg
79,98,Devon Snyder,98,437,Skokie,CurbRamp,42.022621,-87.728439,_KrmBjHlebAncoZc40tsSg,2022-05-10 14:58:11.132-07,t,3.0,f,not enough landing space,NaN,POINT (-87.72844 42.02262),16,img/chicago_with_icons/label_id_98.jpg
82,102,Devon Snyder,102,447,Skokie,CurbRamp,42.023483,-87.728134,BMhU3MUjwcgFnO2ax-6TxQ,2022-05-10 15:00:54.488-07,t,3.0,f,"surface problem,points into traffic",NaN,POINT (-87.72813 42.02348),3,img/chicago_with_icons/label_id_102.jpg


In [50]:
append_img_url(chicago_problem, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
80,99,Devon Snyder,99,447,Skokie,SurfaceProblem,42.022999,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:05.467-07,t,1.0,f,grass,NaN,POINT (-87.72815 42.02300),63,img/chicago_with_icons/label_id_99.jpg
81,100,Devon Snyder,100,447,Skokie,SurfaceProblem,42.023033,-87.728149,2vMz67hXm2ZGNcKHbyoR8g,2022-05-10 14:59:10.286-07,t,1.0,f,NaN,NaN,POINT (-87.72815 42.02303),67,img/chicago_with_icons/label_id_100.jpg
84,105,Devon Snyder,105,447,Skokie,SurfaceProblem,42.023663,-87.728111,spnkMkInfnfp6Se2o7-t9Q,2022-05-10 15:02:43.061-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72811 42.02366),88,img/chicago_with_icons/label_id_105.jpg
85,106,Devon Snyder,106,447,Skokie,SurfaceProblem,42.024178,-87.728104,-YaXYeIUgy5a5bYJcFDMQQ,2022-05-10 15:03:40.542-07,t,1.0,f,"grass,cracks",NaN,POINT (-87.72810 42.02418),1,img/chicago_with_icons/label_id_106.jpg
91,113,Devon Snyder,113,437,Skokie,Obstacle,42.024498,-87.728394,p_-iwPEGdz3zFZzojysq_Q,2022-05-10 15:08:14.409-07,t,4.0,t,construction,NaN,POINT (-87.72839 42.02450),8,img/chicago_with_icons/label_id_113.jpg


In [52]:
append_img_url(chicago_nosidewalk, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
366,500,Devon Snyder,500,445,Robbins,NoSidewalk,41.644047,-87.700142,MpVkKx5iIEW8H1YMT3y6-A,2022-05-13 08:31:14.223-07,NaN,NaN,f,"street has no sidewalks,ends abruptly",NaN,POINT (-87.70014 41.64405),4,img/chicago_with_icons/label_id_500.jpg
602,853,Devon Snyder,853,381,Riverdale,NoSidewalk,41.644432,-87.623962,sGOC_qNLg4cvbj-hYYl3eQ,2022-05-24 10:48:56.3-07,t,1.0,f,"street has a sidewalk,ends abruptly",NaN,POINT (-87.62396 41.64443),5,img/chicago_with_icons/label_id_853.jpg
638,892,Devon Snyder,892,380,Riverdale,NoSidewalk,41.644691,-87.621674,inDdl8tcuankYFQkSoBSRw,2022-05-24 11:03:23.188-07,t,1.0,f,"street has a sidewalk,ends abruptly",NaN,POINT (-87.62167 41.64469),6,img/chicago_with_icons/label_id_892.jpg
1083,1448,Devon Snyder,1448,377,Northbrook,NoSidewalk,42.124695,-87.878914,cbhDyDQGgC4KPYVigFP92Q,2022-05-25 14:18:19.177-07,NaN,2.0,f,"street has a sidewalk,ends abruptly",NaN,POINT (-87.87891 42.12469),3,img/chicago_with_icons/label_id_1448.jpg
1085,1450,Devon Snyder,1450,377,Northbrook,NoSidewalk,42.124557,-87.878883,511McJ_OoIVzVnTQlQ__ZA,2022-05-25 14:19:11.146-07,t,2.0,f,street has a sidewalk,NaN,POINT (-87.87888 42.12456),2,img/chicago_with_icons/label_id_1450.jpg


In [53]:
append_img_url(chicago_occlusion, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
103,125,Devon Snyder,125,404,Skokie,Occlusion,42.025734,-87.728043,dlv0HVOAwKimKu6ukF2_9Q,2022-05-10 15:17:03.745-07,NaN,NaN,f,NaN,NaN,POINT (-87.72804 42.02573),1,img/chicago_with_icons/label_id_125.jpg
104,126,Devon Snyder,126,404,Skokie,Occlusion,42.025826,-87.728050,dlv0HVOAwKimKu6ukF2_9Q,2022-05-10 15:17:06.842-07,NaN,NaN,f,NaN,NaN,POINT (-87.72805 42.02583),2,img/chicago_with_icons/label_id_126.jpg
105,127,Devon Snyder,127,404,Skokie,Occlusion,42.025806,-87.728043,40qnTlab3GmscWzN8Ro8hQ,2022-05-10 15:17:46.151-07,NaN,NaN,f,NaN,NaN,POINT (-87.72804 42.02581),3,img/chicago_with_icons/label_id_127.jpg
106,128,Devon Snyder,128,404,Skokie,Occlusion,42.025879,-87.728065,40qnTlab3GmscWzN8Ro8hQ,2022-05-10 15:17:50.723-07,NaN,NaN,f,NaN,NaN,POINT (-87.72807 42.02588),4,img/chicago_with_icons/label_id_128.jpg
109,131,Devon Snyder,131,404,Skokie,Occlusion,42.025970,-87.728050,YVzVXofwGFkmSjzwZDR-xQ,2022-05-10 15:19:14.238-07,NaN,NaN,f,NaN,NaN,POINT (-87.72805 42.02597),5,img/chicago_with_icons/label_id_131.jpg


In [54]:
append_img_url(chicago_signal, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
114,136,Devon Snyder,136,413,Skokie,Signal,42.026096,-87.728416,l_joW17kVMrbCc-c0wgZOQ,2022-05-10 15:20:08.992-07,t,NaN,f,has button,NaN,POINT (-87.72842 42.02610),7,img/chicago_with_icons/label_id_136.jpg
117,139,Devon Snyder,139,413,Skokie,Signal,42.026192,-87.728462,nG8zZVxy9NoJ2koyn3UHrg,2022-05-10 15:21:23.276-07,NaN,NaN,f,has button,NaN,POINT (-87.72846 42.02619),1,img/chicago_with_icons/label_id_139.jpg
121,143,Devon Snyder,143,404,Skokie,Signal,42.026100,-87.727997,jduryxSb7BLpIXL5HaeGog,2022-05-10 15:22:15.718-07,t,NaN,f,has button,NaN,POINT (-87.72800 42.02610),3,img/chicago_with_icons/label_id_143.jpg
125,147,Devon Snyder,147,413,Skokie,Signal,42.026367,-87.728401,ysZ2ATJKeGS2idQpiyp1gA,2022-05-10 15:23:40.32-07,t,NaN,f,has button,NaN,POINT (-87.72840 42.02637),6,img/chicago_with_icons/label_id_147.jpg
128,150,Devon Snyder,150,413,Skokie,Signal,42.026443,-87.728386,wxy58lr0IsL44sQE9B9UWg,2022-05-10 15:24:49.83-07,t,NaN,f,has button,NaN,POINT (-87.72839 42.02644),5,img/chicago_with_icons/label_id_150.jpg


In [55]:
append_img_url(chicago_crosswalk, chicago_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
115,137,Devon Snyder,137,404,Skokie,Crosswalk,42.026157,-87.728172,l_joW17kVMrbCc-c0wgZOQ,2022-05-10 15:20:17.331-07,t,3.0,f,"uneven surface,paint fading",NaN,POINT (-87.72817 42.02616),5,img/chicago_with_icons/label_id_137.jpg
123,145,Devon Snyder,145,405,Skokie,Crosswalk,42.026199,-87.728035,jduryxSb7BLpIXL5HaeGog,2022-05-10 15:22:40.88-07,t,3.0,f,paint fading,NaN,POINT (-87.72803 42.02620),2,img/chicago_with_icons/label_id_145.jpg
124,146,Devon Snyder,146,413,Skokie,Crosswalk,42.026226,-87.728371,jduryxSb7BLpIXL5HaeGog,2022-05-10 15:22:52.098-07,t,3.0,f,"uneven surface,paint fading,broken surface",NaN,POINT (-87.72837 42.02623),4,img/chicago_with_icons/label_id_146.jpg
130,152,Devon Snyder,152,438,Skokie,Crosswalk,42.026356,-87.728233,wxy58lr0IsL44sQE9B9UWg,2022-05-10 15:25:09.512-07,t,2.0,f,paint fading,NaN,POINT (-87.72823 42.02636),3,img/chicago_with_icons/label_id_152.jpg
143,165,Devon Snyder,165,412,Skokie,Crosswalk,42.026150,-87.729416,ztmynHmdiMkTfNFKOxPu2A,2022-05-11 08:07:55.833-07,t,3.0,f,paint fading,NaN,POINT (-87.72942 42.02615),1,img/chicago_with_icons/label_id_165.jpg


In [15]:
#get center location of the geodf
chicago_curb_ramp.dissolve().centroid

0    POINT (-87.76274 41.93642)
dtype: geometry

# Seattle

In [59]:
#get curb ramp cluster dataset for seattle 
seattle_curb_ramp = cluster_label_type_at_index(seattle,0)[2]
#get problem cluster dataset for seattle 
seattle_problem = cluster_label_type_at_index(seattle,2)[2]
#get no curb cluster dataset for seattle 
seattle_nocurb = cluster_label_type_at_index(seattle,7)[2]
#get no sidewalk cluster dataset for seattle 
seattle_nosidewalk = cluster_label_type_at_index(seattle,1)[2]
#get occlusion cluster dataset for seattle 
seattle_occlusion = cluster_label_type_at_index(seattle,3)[2]
#get signal cluster dataset for seattle 
seattle_signal = cluster_label_type_at_index(seattle,9)[2]
#get crosswalk cluster dataset for seattle 
seattle_crosswalk = cluster_label_type_at_index(seattle,8)[2]

In [17]:
# locate folder with images
seattle_folder ="seattle_with_icons/"

In [18]:
append_img_url(seattle_curb_ramp, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
10,19,Devon Snyder,19,482,University District,CurbRamp,47.661415,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:11.854-07,t,3.0,f,"narrow,not enough landing space,surface problem",NaN,POINT (-122.32083 47.66142),34,img/seattle_with_icons/label_id_19.jpg
12,21,Devon Snyder,21,482,University District,CurbRamp,47.661472,-122.320847,xx0-vgC0WHpTn2feGbrbUA,2022-05-10 14:09:32.271-07,t,2.0,f,not enough landing space,NaN,POINT (-122.32085 47.66147),33,img/seattle_with_icons/label_id_21.jpg
13,22,Devon Snyder,22,482,University District,CurbRamp,47.661488,-122.320854,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:09:52.251-07,t,2.0,f,"surface problem,not enough landing space",NaN,POINT (-122.32085 47.66149),32,img/seattle_with_icons/label_id_22.jpg
15,27,Devon Snyder,27,483,University District,CurbRamp,47.661472,-122.320702,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:10:59.364-07,t,3.0,f,not level with street,NaN,POINT (-122.32070 47.66147),31,img/seattle_with_icons/label_id_27.jpg
28,41,Devon Snyder,41,486,University District,CurbRamp,47.661228,-122.319885,hr-qZOb3MdbSB1ZNBjsSIw,2022-05-10 14:18:55.439-07,t,2.0,f,surface problem,a bit far away even zoomed in but tried best to rate,POINT (-122.31989 47.66123),41,img/seattle_with_icons/label_id_41.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2489,mikey,2489,810,Windermere,CurbRamp,47.672092,-122.267960,09WtcNVwH9KSZ8B4Hqm3Bg,2022-07-06 14:12:02.237-07,NaN,2.0,f,surface problem,NaN,POINT (-122.26796 47.67209),5,img/seattle_with_icons/label_id_2489.jpg
2010,2528,Devon Snyder,2528,487,University District,CurbRamp,47.659523,-122.319687,jCGaqx_i5lllqbp3q1GWUw,2022-11-09 08:03:27.714-08,NaN,3.0,f,"missing tactile warning,points into traffic,narrow",NaN,POINT (-122.31969 47.65952),4,img/seattle_with_icons/label_id_2528.jpg
2011,2529,Devon Snyder,2529,488,University District,CurbRamp,47.659401,-122.319725,z9UEjhLFWd_yO6TmMzABfA,2022-11-09 08:04:02.134-08,NaN,2.0,f,"points into traffic,missing tactile warning",NaN,POINT (-122.31973 47.65940),3,img/seattle_with_icons/label_id_2529.jpg
2012,2530,Devon Snyder,2530,476,University District,CurbRamp,47.659454,-122.319939,ndtOrb8y8Cnt1HpvVXWNuQ,2022-11-09 08:04:41.577-08,NaN,2.0,f,"narrow,missing tactile warning",NaN,POINT (-122.31994 47.65945),2,img/seattle_with_icons/label_id_2530.jpg


In [19]:
append_img_url(seattle_problem, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
0,3,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,POINT (-122.32071 47.66104),240,img/seattle_with_icons/label_id_3.jpg
1,4,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,POINT (-122.32070 47.66103),238,img/seattle_with_icons/label_id_4.jpg
2,5,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,POINT (-122.32071 47.66106),94,img/seattle_with_icons/label_id_5.jpg
4,9,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,POINT (-122.32066 47.66110),237,img/seattle_with_icons/label_id_9.jpg
18,30,Devon Snyder,30,483,University District,Obstacle,47.661419,-122.320488,iI69DtsYap_Fjiz-RZ4i1A,2022-05-10 14:13:05.844-07,f,2.0,f,pole,NaN,POINT (-122.32049 47.66142),113,img/seattle_with_icons/label_id_30.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,2493,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,POINT (-122.26793 47.67204),6,img/seattle_with_icons/label_id_2493.jpg
2006,2494,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,POINT (-122.26810 47.67193),5,img/seattle_with_icons/label_id_2494.jpg
2008,2496,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,POINT (-122.26818 47.67188),4,img/seattle_with_icons/label_id_2496.jpg
2016,2534,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,POINT (-122.31992 47.65972),2,img/seattle_with_icons/label_id_2534.jpg


In [29]:
#get center location of the geodf
seattle.dissolve().centroid

0    POINT (-122.33162 47.61003)
dtype: geometry

In [31]:
append_img_url(seattle_nocurb, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
154,205,Devon Snyder,205,179,International District,NoCurbRamp,47.600964,-122.334297,BsqrOdDSbJWA6WB0NVJeiA,2022-05-11 08:53:02.452-07,t,3.0,f,alternate route present,NaN,POINT (-122.33430 47.60096),78,img/seattle_with_icons/label_id_205.jpg
214,270,Devon Snyder,270,258,International District,NoCurbRamp,47.600143,-122.334801,Pv9d691EbTY_Hy83GaEerw,2022-05-11 09:25:17.111-07,t,5.0,f,no alternate route,NaN,POINT (-122.33480 47.60014),74,img/seattle_with_icons/label_id_270.jpg
216,272,Devon Snyder,272,258,International District,NoCurbRamp,47.599949,-122.334808,Pv9d691EbTY_Hy83GaEerw,2022-05-11 09:25:50.974-07,t,5.0,f,no alternate route,NaN,POINT (-122.33481 47.59995),75,img/seattle_with_icons/label_id_272.jpg
276,337,Devon Snyder,337,272,Queen Anne,NoCurbRamp,47.646950,-122.360657,YpqnWy1iKkotWCA4QFgpQw,2022-05-12 08:25:08.382-07,t,2.0,f,no alternate route,NaN,POINT (-122.36066 47.64695),72,img/seattle_with_icons/label_id_337.jpg
277,338,Devon Snyder,338,296,Queen Anne,NoCurbRamp,47.646812,-122.360641,IIX2_fOXHytzdLB_P4tm4Q,2022-05-12 08:25:41.514-07,t,3.0,f,no alternate route,NaN,POINT (-122.36064 47.64681),73,img/seattle_with_icons/label_id_338.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,2467,mikey,2467,810,Windermere,NoCurbRamp,47.672066,-122.266052,j4WcLDZg7JJkQCvEnux3uA,2022-07-06 13:49:01.93-07,NaN,3.0,f,alternate route present,NaN,POINT (-122.26605 47.67207),6,img/seattle_with_icons/label_id_2467.jpg
1980,2468,mikey,2468,810,Windermere,NoCurbRamp,47.672081,-122.266296,j4WcLDZg7JJkQCvEnux3uA,2022-07-06 13:49:07.856-07,NaN,3.0,f,alternate route present,NaN,POINT (-122.26630 47.67208),4,img/seattle_with_icons/label_id_2468.jpg
1981,2469,mikey,2469,810,Windermere,NoCurbRamp,47.672173,-122.266289,j4WcLDZg7JJkQCvEnux3uA,2022-07-06 13:49:11.491-07,NaN,5.0,f,no alternate route,NaN,POINT (-122.26629 47.67217),113,img/seattle_with_icons/label_id_2469.jpg
1982,2470,mikey,2470,810,Windermere,NoCurbRamp,47.672184,-122.266266,j4WcLDZg7JJkQCvEnux3uA,2022-07-06 13:49:15.006-07,NaN,5.0,f,no alternate route,NaN,POINT (-122.26627 47.67218),2,img/seattle_with_icons/label_id_2470.jpg


In [32]:
append_img_url(seattle_nosidewalk, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
174,226,Devon Snyder,226,243,International District,NoSidewalk,47.601002,-122.335220,elIi0Q0khZq587iJWROCaQ,2022-05-11 09:04:05.179-07,t,5.0,t,NaN,construction,POINT (-122.33522 47.60100),61,img/seattle_with_icons/label_id_226.jpg
192,248,Devon Snyder,248,182,International District,NoSidewalk,47.600845,-122.335861,26SL656wwKAlCUncgY1Idg,2022-05-11 09:15:15.794-07,NaN,5.0,t,NaN,"construction, alternative route available",POINT (-122.33586 47.60085),62,img/seattle_with_icons/label_id_248.jpg
193,249,Devon Snyder,249,249,International District,NoSidewalk,47.600681,-122.335953,CzQAnKygyu-2QlEOsq-a1Q,2022-05-11 09:15:40.975-07,NaN,5.0,t,NaN,"construction, alternative route available",POINT (-122.33595 47.60068),63,img/seattle_with_icons/label_id_249.jpg
524,694,Devon Snyder,694,715,Windermere,NoSidewalk,47.672234,-122.268181,65FQ-Ikjf8gSpzEoqndvoQ,2022-05-13 10:11:02.091-07,t,1.0,f,street has a sidewalk,NaN,POINT (-122.26818 47.67223),7,img/seattle_with_icons/label_id_694.jpg
877,1214,Devon Snyder,1214,136,Delridge,NoSidewalk,47.553738,-122.361618,Z-nc2QMpJZSUQ-aDZSmqsA,2022-05-25 09:34:51.712-07,t,5.0,f,street has no sidewalks,NaN,POINT (-122.36162 47.55374),20,img/seattle_with_icons/label_id_1214.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,2339,mikey,2339,541,Greenwood,NoSidewalk,47.694859,-122.349899,TYibH5wbuF7P6g2O3zuqiA,2022-07-05 12:14:20.793-07,NaN,5.0,f,"street has no sidewalks,ends abruptly",NaN,POINT (-122.34990 47.69486),8,img/seattle_with_icons/label_id_2339.jpg
2003,2491,mikey,2491,851,Windermere,NoSidewalk,47.672295,-122.268089,0Z8txi3E8Fg2UtsmtHCLVQ,2022-07-06 14:12:23.995-07,NaN,3.0,f,street has a sidewalk,NaN,POINT (-122.26809 47.67229),7,img/seattle_with_icons/label_id_2491.jpg
2004,2492,mikey,2492,715,Windermere,NoSidewalk,47.672153,-122.268234,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:31.286-07,NaN,3.0,f,street has a sidewalk,NaN,POINT (-122.26823 47.67215),6,img/seattle_with_icons/label_id_2492.jpg
2007,2495,mikey,2495,715,Windermere,NoSidewalk,47.672016,-122.268433,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:48.626-07,NaN,3.0,f,street has a sidewalk,NaN,POINT (-122.26843 47.67202),4,img/seattle_with_icons/label_id_2495.jpg


In [60]:
append_img_url(seattle_occlusion, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
3,8,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,POINT (-122.32072 47.66111),4,img/seattle_with_icons/label_id_8.jpg
11,20,Devon Snyder,20,480,University District,Occlusion,47.661198,-122.320717,YFf5aHrB0DbjwI00YDfZiw,2022-05-10 14:09:07.462-07,NaN,NaN,f,NaN,NaN,POINT (-122.32072 47.66120),5,img/seattle_with_icons/label_id_20.jpg
17,29,Devon Snyder,29,480,University District,Occlusion,47.661171,-122.320709,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:12:03.873-07,NaN,NaN,f,NaN,NaN,POINT (-122.32071 47.66117),6,img/seattle_with_icons/label_id_29.jpg
24,36,Devon Snyder,36,486,University District,Occlusion,47.661209,-122.320244,l9zMoY0tPEhjtAftJ-XzYg,2022-05-10 14:16:00.723-07,NaN,NaN,f,NaN,NaN,POINT (-122.32024 47.66121),7,img/seattle_with_icons/label_id_36.jpg
39,57,Devon Snyder,57,479,University District,Occlusion,47.660923,-122.319878,qY3nCKr-ACA7suPZuH5Kqg,2022-05-10 14:25:45.438-07,NaN,NaN,f,NaN,NaN,POINT (-122.31988 47.66092),8,img/seattle_with_icons/label_id_57.jpg


In [61]:
append_img_url(seattle_signal, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
5,10,Devon Snyder,10,481,University District,Signal,47.661236,-122.320930,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:03:18.775-07,t,NaN,f,"button waist height,has button",NaN,POINT (-122.32093 47.66124),5,img/seattle_with_icons/label_id_10.jpg
9,18,Devon Snyder,18,482,University District,Signal,47.661438,-122.320831,j3GefSTHZmrn2LHTGg8SFA,2022-05-10 14:08:04.76-07,t,NaN,f,"has button,button waist height",NaN,POINT (-122.32083 47.66144),11,img/seattle_with_icons/label_id_18.jpg
14,23,Devon Snyder,23,481,University District,Signal,47.661495,-122.320953,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:10:08.143-07,t,NaN,f,"button waist height,has button",NaN,POINT (-122.32095 47.66150),12,img/seattle_with_icons/label_id_23.jpg
202,258,Devon Snyder,258,244,International District,Signal,47.599884,-122.335556,n1cMOes1ASyujsdkgArkSg,2022-05-11 09:22:09.508-07,t,NaN,f,"button waist height,has button",NaN,POINT (-122.33556 47.59988),9,img/seattle_with_icons/label_id_258.jpg
203,259,Devon Snyder,259,254,International District,Signal,47.600208,-122.335564,n1cMOes1ASyujsdkgArkSg,2022-05-11 09:22:18.945-07,t,NaN,f,"has button,button waist height",NaN,POINT (-122.33556 47.60021),7,img/seattle_with_icons/label_id_259.jpg


In [64]:
append_img_url(seattle_crosswalk, seattle_folder)

,Unnamed: 0,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,geometry,cluster_id,img_url
6,14,Devon Snyder,14,482,University District,Crosswalk,47.661316,-122.320869,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:04:24.805-07,NaN,2.0,f,NaN,NaN,POINT (-122.32087 47.66132),9,img/seattle_with_icons/label_id_14.jpg
7,15,Devon Snyder,15,480,University District,Crosswalk,47.661205,-122.320778,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:04:36.076-07,t,2.0,f,paint fading,NaN,POINT (-122.32078 47.66121),8,img/seattle_with_icons/label_id_15.jpg
16,28,Devon Snyder,28,483,University District,Crosswalk,47.661461,-122.320740,qCkbp2lszdGrR5s3hdSiTQ,2022-05-10 14:11:19.541-07,t,3.0,f,paint fading,NaN,POINT (-122.32074 47.66146),7,img/seattle_with_icons/label_id_28.jpg
148,198,Devon Snyder,198,179,International District,Crosswalk,47.600964,-122.334213,1mcZ_CqLAp3r4RPkBcqWYg,2022-05-11 08:50:38.818-07,t,2.0,f,NaN,NaN,POINT (-122.33421 47.60096),16,img/seattle_with_icons/label_id_198.jpg
150,200,Devon Snyder,200,208,International District,Crosswalk,47.600895,-122.334084,BsqrOdDSbJWA6WB0NVJeiA,2022-05-11 08:51:17.737-07,t,2.0,f,NaN,NaN,POINT (-122.33408 47.60089),17,img/seattle_with_icons/label_id_200.jpg


## Seattle

In [42]:
seattle_problem["clustered"] = seattle_problem.duplicated(subset="cluster_id", keep=False)

In [43]:
seattle_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_problem.json', driver="GeoJSON") 

In [33]:
seattle_nosidewalk["clustered"] = seattle_nosidewalk.duplicated(subset="cluster_id", keep=False)

In [34]:
seattle_nosidewalk.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_nosidewalk.json', driver="GeoJSON") 

In [35]:
seattle_nocurb["clustered"] = seattle_nocurb.duplicated(subset="cluster_id", keep=False)

In [36]:
seattle_nocurb.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_nocurb.json', driver="GeoJSON") 

In [44]:
seattle_curb_ramp["clustered"] = seattle_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [45]:
seattle_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_curb_ramp.json', driver="GeoJSON") 

In [65]:
seattle_occlusion["clustered"] = seattle_occlusion.duplicated(subset="cluster_id", keep=False)
seattle_occlusion.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_occlusion.json', driver="GeoJSON") 

In [66]:
seattle_crosswalk["clustered"] = seattle_crosswalk.duplicated(subset="cluster_id", keep=False)
seattle_crosswalk.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_crosswalk.json', driver="GeoJSON") 

In [67]:
seattle_signal["clustered"] = seattle_signal.duplicated(subset="cluster_id", keep=False)
seattle_signal.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/seattle_signal.json', driver="GeoJSON") 

## Chicago

In [46]:
chicago_problem["clustered"] = chicago_problem.duplicated(subset="cluster_id", keep=False)

In [47]:
chicago_problem.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_problem.json', driver="GeoJSON") 

In [48]:
chicago_curb_ramp["clustered"] = chicago_curb_ramp.duplicated(subset="cluster_id", keep=False)

In [49]:
chicago_curb_ramp.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_curb_ramp.json', driver="GeoJSON") 

In [28]:
chicago_nosidewalk["clustered"] = chicago_nosidewalk.duplicated(subset="cluster_id", keep=False)

In [29]:
chicago_nosidewalk.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_nosidewalk.json', driver="GeoJSON") 

In [56]:
chicago_occlusion["clustered"] = chicago_occlusion.duplicated(subset="cluster_id", keep=False)
chicago_occlusion.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_occlusion.json', driver="GeoJSON") 

In [57]:
chicago_crosswalk["clustered"] = chicago_crosswalk.duplicated(subset="cluster_id", keep=False)
chicago_crosswalk.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_crosswalk.json', driver="GeoJSON") 

In [58]:
chicago_signal["clustered"] = chicago_signal.duplicated(subset="cluster_id", keep=False)
chicago_signal.to_file('/Users/chuli/Documents/GitHub/ps-validation-study.github.io/data/chicago_signal.json', driver="GeoJSON") 